In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from IPython.display import display, HTML
import time

In [4]:
HR = pd.read_csv("./HR_50.csv")

In [5]:
def processData(data, feature): 
    Y = data[feature]
    X = data.drop(feature, axis = 1)
    X = pd.get_dummies(X)
    X = preprocessing.scale(X)
    X = pd.DataFrame(X)
    return X, Y

In [6]:
def write_scores(filename, x, y):
    file = open(filename, "w")
    if x is not None:
        for i in range(0, len(x)):
            file.write(str(x[i]) + ", ")
            file.write(str(y[i]) + "\n")
        file.write("\n")
    file.close()

In [7]:
def write_scores(filename, x, y, a, b):
    file = open(filename, "w")
    if x is not None:
        for i in range(0, len(x)):
            file.write(str(x[i]) + ", ")
            file.write(str(y[i]) + "\n")
        file.write("\n")
    file.write("\n")
    if a is not None:
        for i in range(0, len(a)):
            file.write(str(a[i]) + ", ")
            file.write(str(b[i]) + "\n")
        file.write("\n")
    file.close()

In [63]:
# data size vs accuracy
HR_dt = DecisionTreeClassifier(max_depth=30)
HR_means = []
HR_std = []
HR_mean_training = []
training_time = []
print "start"
for i in range(1, 51):
    HR_sample = HR.sample(HR["Age"].count() / 50 * i)
    HR_X, HR_Y = processData(HR_sample, "JobSatisfaction")
    HR_scores = cross_val_score(HR_dt, HR_X, HR_Y, cv = 5)
    start = time.time()
    HR_dt.fit(HR_X, HR_Y)
    end = time.time()
    HR_means.append(HR_scores.mean())
    HR_std.append(HR_scores.std())
    HR_mean_training.append(HR_dt.score(HR_X, HR_Y))
    training_time.append(end-start)
print "done"


start


done


In [64]:
#max depth
HR_means_leaf = []
HR_std_leaf = []
HR_mean_training_leaf = []
print "start"
for i in range(1, 11):
    HR_dt = DecisionTreeClassifier(max_depth=i*5)
    # HR_sample = HR.sample(HR["HR"].count() / 51 * (i - 1))
    HR_X, HR_Y = processData(HR, "JobSatisfaction")
    HR_scores = cross_val_score(HR_dt, HR_X, HR_Y, cv = 10)
    HR_means_leaf.append(HR_scores.mean())
    HR_std_leaf.append(HR_scores.std())
    HR_dt.fit(HR_X, HR_Y)
    HR_mean_training_leaf.append(HR_dt.score(HR_X, HR_Y))
    print i
print "done"

start
1


2
3


4
5


6


7


8
9


10
done


In [65]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 8)
x = []
print "start"
for i in list(range(1,51)) :
    x.append(i*2)
HR_graph = sns.pointplot(x, HR_means, ax = ax, color = 'r')
# HR_graph = plt.errorbar(x, HR_mean_training, HR_std, linestyle='None', marker='^')
# sns.pointplot(x, HR_mean_training, ax = ax, color = 'r')

HR_graph.set_xlabel("% of training data")
HR_graph.set_ylabel("% accuracy")
HR_graph.set_title("HR Set, Decision Tree, % of training data vs % accuracy")
plt.savefig("HR_DT_Training_Data_Accuracy")
write_scores("HR_DT_Training_Data_Accuracy.csv", x, HR_means, None, None)
plt.show()


start


In [66]:

fig2, ax2 = plt.subplots()
fig2.set_size_inches(15, 8)
x = []
for i in list(range(1,11)) :
    x.append(i*5)
print len(x)
print len(HR_mean_training_leaf)
HR_graph_leaf = sns.pointplot(x, HR_means_leaf, ax = ax2)
sns.pointplot(x, HR_means_leaf, ax = ax2, color = 'r')
HR_graph_leaf.set_xlabel("# of leaf nodes")
HR_graph_leaf.set_ylabel("% accuracy")
HR_graph_leaf.set_title("HR Set, Decision Tree, # of leaf nodes vs % accuracy")
plt.savefig("HR_DT_LeafNodes_Accuracy")
write_scores("HR_DT_LeafNodes_Accuracy.csv", x, HR_means_leaf, None, None)
write_scores("HR_DT_LeafNodes_Accuracy_training.csv", x, HR_mean_training_leaf, None, None)
write_scores("HR_DT_training_time_data_size.csv", x, training_time, None, None)
plt.show()
print "done"

10
10
done


#Boosting

In [67]:
file = open("HR_adaboost_max_depth_results.csv", "w")
print("Beginning model complexity analysis for AdaBoost... max_depth")
file.write("max_depth" + ", " + "cross_val_score" + ", " + "training_score" + ", " + "testing_score" + ", training_time\n")
for i in range(1, 11):
    classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=i * 5), n_estimators=50)
    result = ""
    input_list, output_list = processData(HR, "JobSatisfaction")
    result += (str(i * 5) + "," + str(cross_val_score(
    classifier, input_list, output_list).mean()) + ", ")
    start = time.time()
    classifier.fit(input_list, output_list)
    end = time.time()
    result += str(classifier.score(input_list, output_list)) + ", " + str(end-start) + "\n"
    print(result)
    file.write(result)
file.close()

Beginning model complexity analysis for AdaBoost... max_depth


5,0.5100888797203353, 1.0, 0.452000141144



10,0.55569775596818, 1.0, 0.652000188828



15,0.5563738716282934, 1.0, 0.705999851227



20,0.5236512699282521, 1.0, 0.0169999599457

25,0.5202554513156746, 1.0, 0.0160000324249

30,0.5236568118598924, 1.0, 0.0169999599457

35,0.5188949070979876, 1.0, 0.0159997940063



40,0.5209357234245182, 1.0, 0.0160000324249

45,0.5182146349891441, 1.0, 0.0160000324249

50,0.5188949070979876, 1.0, 0.0160000324249



#KNN

In [68]:
HR_knn = KNeighborsClassifier(n_neighbors = 5)
HR_means_KNN = []
HR_std_KNN = []
HR_mean_training_KNN = []
training_time = []
print "start"
for i in range(1, 11):
    HR_sample = HR.sample(HR["JobSatisfaction"].count() / 10 * i)
    HR_X, HR_Y = processData(HR_sample, "JobSatisfaction")
    HR_scores = cross_val_score(HR_knn, HR_X, HR_Y, cv = 10)
    start = time.time()
    HR_knn.fit(HR_X, HR_Y)
    end = time.time()
    training_time.append(end-start)
    HR_means_KNN.append(HR_scores.mean())
    HR_std_KNN.append(HR_scores.std())
    HR_mean_training_KNN.append(HR_knn.score(HR_X, HR_Y))
    print i
print "done"

start
1
2
3


4
5


6


7


8


9


10
done


In [70]:
HR_means_KNN_neighbors = []
HR_std_KNN_neighbors = []
HR_mean_training_KNN_neighbors = []
print "start"
for i in range(1, 51, 5):
    HR_knn = KNeighborsClassifier(n_neighbors = i)
    HR_sample = HR.sample(HR["JobSatisfaction"].count() / 10)
    HR_X, HR_Y = processData(HR, "JobSatisfaction")
    HR_scores = cross_val_score(HR_knn, HR_X, HR_Y, cv = 10)
    HR_knn.fit(HR_X, HR_Y)
    HR_means_KNN_neighbors.append(HR_scores.mean())
    HR_std_KNN_neighbors.append(HR_scores.std())
    print "start training set"
    HR_mean_training_KNN_neighbors.append(HR_knn.score(HR_X, HR_Y))
    print i
print "done"

start


start training set
1


start training set
6


start training set
11


start training set
16


...

start training set
31


start training set
36


start training set


41


start training set
46
done


In [70]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 8)
x = []
# for i in list(range(1,51)) :
#     x.append(i*2)
# HR_graph = sns.pointplot(x, HR_means, ax = ax)
# sns.pointplot(x, HR_mean_training, ax = ax, color = 'r')
# HR_graph.set_xlabel("% of training data")
# HR_graph.set_ylabel("% accuracy")
# HR_graph.set_title("HR Set, Decision Tree, % of training data vs % accuracy")
# plt.show()
for i in list(range(1,11)) :
    x.append(i * 10)
HR_graph = sns.pointplot(x, HR_means_KNN, ax = ax)
sns.pointplot(x, HR_mean_training_KNN, ax = ax, color = 'r')
HR_graph.set_xlabel("% training data")
HR_graph.set_ylabel("% accuracy")
HR_graph.set_title("HR Set, KNN, % training data vs % accuracy")
plt.savefig('HR Set_KNN_Training_Data_Vs_Accuracy.png', bbox_inches='tight')
write_scores("HR Set_KNN_Training_Data_Vs_Accuracy.csv", x, HR_means_KNN, x, HR_mean_training_KNN)
write_scores("HR_KNN_TrainingSpeed.csv", x, training_time, None, None)
plt.show()
print "done"

done


In [73]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 8)
x = []
for i in list(range(1,11)) :
    x.append(i * 3)
HR_graph = sns.pointplot(x, HR_means_KNN_neighbors, ax = ax)
sns.pointplot(x, HR_means_KNN_neighbors, ax = ax, color = 'r')
HR_graph.set_xlabel("Number of neighbors")
HR_graph.set_ylabel("% accuracy")
HR_graph.set_title("HR Set, KNN, neighbors vs % accuracy")
plt.savefig("HR_KNN_Neighbors_Accuracy.png")
write_scores("HR_KNN_Neighbors_Accuracy.csv", x, HR_means_KNN_neighbors, x, HR_mean_training_KNN_neighbors)
plt.show()
print "done"

done


In [75]:
HR_MLP = MLPClassifier()
HR_means_NN = []
HR_means_NN_training = []
fit_time = []
score_time = []
for i in range(1, 5) :
    print "start"
    HR_X_train = HR_X.sample(frac=0.2 * i, random_state=1)
    HR_X_test = HR_X.drop(HR_X_train.index)
    HR_Y_train = HR_Y[HR_X_train.index]
    HR_Y_test = HR_Y.drop(HR_Y_train.index)
    start = time.time()
    HR_MLP.fit(HR_X_train, HR_Y_train)
    end = time.time()
    print(end - start)
    fit_time.append(end-start)
    score = HR_MLP.score(HR_X_test, HR_Y_test)
    end2 = time.time()
    print(end2 - end)
    score_time.append(end2-end)
    print score
    HR_means_NN.append(score)
    score = HR_MLP.score(HR_X_train, HR_Y_train)
    HR_means_NN_training.append(score)
    print score
# print "start"
# for i in range(1, 5):
#     HR_sample = HR.sample(HR["HR"].count() / (20 * 5) * i)
#     HR_X, HR_Y = processData(HR_sample, "HR")
#     HR_scores = cross_val_score(HR_dt, HR_X, HR_Y, cv = 10)
#     HR_dt.fit(HR_X, HR_Y)
#     HR_means.append(HR_scores.mean())
#     HR_std.append(HR_scores.std())
#     HR_mean_training.append(HR_dt.score(HR_X, HR_Y))
# print "done"

start


C:\Users\Wilche\Desktop\Machine_Learning\assignment1\conda\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.743999958038
0.00799989700317
0.5476190476190477
1.0
start


1.45899987221
0.00600004196167
0.5589569160997733
1.0
start


2.1819999218
0.00500011444092
0.5612244897959183
0.9977324263038548
start


2.88900017738
0.00299978256226
0.5714285714285714
0.9982993197278912


In [79]:
HR_MLP = MLPClassifier()
HR_means_NN_layers = []
HR_means_NN_training_layers = []
HR_X_train = HR_X.sample(frac=0.8, random_state=1)
HR_X_test = HR_X.drop(HR_X_train.index)
HR_Y_train = HR_Y[HR_X_train.index]
HR_Y_test = HR_Y.drop(HR_Y_train.index)
for i in range(1, 11) :
    test_scores = []
    train_scores = []
    layers = [i * 20]
    HR_MLP = MLPClassifier(learning_rate_init = float(i) / 50)
    for j in range(1, 5):
        start = time.time()
        HR_MLP.fit(HR_X_train, HR_Y_train)
        end = time.time()
        test_score = HR_MLP.score(HR_X_test, HR_Y_test)
        train_score = HR_MLP.score(HR_X_train, HR_Y_train)
        test_scores.append(test_score)
        end2 = time.time()
        train_scores.append(train_score)
        
    HR_means_NN_layers.append(np.mean(test_score))
    HR_means_NN_training_layers.append(np.mean(train_score))
    print "done"

done


done


done


done


done


done


done


done


done


done


In [79]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 8)
x = []
for i in list(range(1,5)) :
    x.append(i * .2)
HR_graph = sns.pointplot(x, HR_means_NN, ax = ax)
sns.pointplot(x, HR_means_NN_training, ax = ax, color = 'r')
HR_graph.set_xlabel("% training data")
HR_graph.set_ylabel("% accuracy")
HR_graph.set_title("HR Set, NN, % training data vs % accuracy")
write_scores("HR_NN_training_data_accuracy.csv", x, HR_means_NN, x, HR_means_NN_training)
write_scores("HR_NN_training_accuracy_time.csv", x, fit_time, x, score_time)
plt.savefig("HR_NN_training_accuracy_accuracy.png")
plt.show()
print "done"

done


In [80]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 8)
x = []
for i in list(range(1,11)) :
    x.append(float(i)/20)
HR_graph = sns.pointplot(x, HR_means_NN_layers, ax = ax)
sns.pointplot(x, HR_means_NN_training_layers, ax = ax, color = 'r')
HR_graph.set_xlabel("learning rate")
HR_graph.set_ylabel("% accuracy")
HR_graph.set_title("HR Set, NN, learning rate vs % accuracy")
plt.savefig("HR_NN_Learning_Accuracy.png")
write_scores("HR_NN_Learning_Accuracy.csv", x, HR_means_NN_layers, x, HR_means_NN_training_layers)
plt.show()
print "done"

done


In [8]:
def runSVM(kernel, runs, data, feature):
    score = []
    score_train = []
    data_X, data_Y = processData(data, feature)
    data_X_train = data_X.sample(frac=0.8, random_state=1)
    # print(data_X_train.index)
    data_X_test = data_X.drop(data_X_train.index)
    data_Y_train = data_Y[data_X_train.index]
    data_Y_test = data_Y.drop(data_Y_train.index)
    for i in range(1, runs): 
        print i
        SVM = SVC(kernel=kernel)
        print SVM.kernel
        SVM.fit(data_X_train, data_Y_train)
        score.append(SVM.score(data_X_test, data_Y_test))
        score_train.append(SVM.score(data_X_train, data_Y_train))
    return np.mean(score), np.mean(score_train), 

In [9]:
HR_SVM = SVC()
HR_means_SVM = []
HR_std_SVM = []
HR_mean_SVM_training = []
time_run = []
print "start"
for i in range(1, 11):
    HR_sample = HR.sample(HR["JobSatisfaction"].count() / 10 * i)
    HR_X, HR_Y = processData(HR_sample, "JobSatisfaction")
    HR_scores = cross_val_score(HR_SVM, HR_X, HR_Y, cv = 10)
    start = time.time()
    HR_SVM.fit(HR_X, HR_Y)
    end = time.time()
    time_run.append(end - start)
    HR_means_SVM.append(HR_scores.mean())
    HR_std_SVM.append(HR_scores.std())
    HR_mean_SVM_training.append(HR_SVM.score(HR_X, HR_Y))
    print i
print "done"

start
1
2


3


4


5


6


7


8


KeyboardInterrupt: 

In [12]:
linear_score, linear_score_train = runSVM('linear', 5, HR, "JobSatisfaction")
poly_score, poly_score_train = runSVM('poly', 5, HR, "JobSatisfaction")
rbf_score, rbf_score_train = runSVM('rbf', 5, HR, "JobSatisfaction")
sigmoid_score, sigmoid_score_train = runSVM('sigmoid', 5, HR, "JobSatisfaction")


1
linear
2
linear


3
linear


4
linear


1
poly
2
poly


3
poly
4
poly


...

2
rbf


3
rbf


4
rbf


1
sigmoid
2
sigmoid


3
sigmoid
4
sigmoid


In [13]:
SVM_df = pd.DataFrame({'linear': [linear_score, linear_score_train], 'poly': [poly_score, poly_score_train], 'rbf': [rbf_score, rbf_score_train], 'sigmoid': [sigmoid_score, sigmoid_score_train]})
display(SVM_df)
file = open("HR_SVM_kernel.csv", "w")
file.write("linear_score, linear_train, poly_score, poly_train, rbf_score, rbf_train, sigmoid_score, sigmoid_train\n")
file.write(str(linear_score) + ", " + str(linear_score_train) + ", " + str(poly_score) + ", " 
           + str(poly_score_train) + ", " + str(rbf_score) + ", " + str(rbf_score_train) + ',' + str(sigmoid_score) + "," + str(sigmoid_score_train))
file.close()

,linear,poly,rbf,sigmoid
0,0.568027,0.557823,0.561224,0.571429
1,0.624150,0.783163,0.725340,0.607143


In [89]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 8)
x = []
for i in list(range(1,11)) :
    x.append(format(i * .1, '.2f'))
print x
print HR_means_SVM
HR_graph = sns.pointplot(x, HR_means_SVM, ax = ax)
sns.pointplot(x, HR_mean_SVM_training, ax = ax, color = 'r')
HR_graph.set_xlabel("% training data")
HR_graph.set_ylabel("% accuracy")
HR_graph.set_title("HR Set, SVM, % training data vs % accuracy")
plt.savefig("HR_SVM_training_accuracy.png")
write_scores("HR_SVM_training_accuracy.csv", x, HR_means_SVM, x, HR_mean_SVM_training)
write_scores("HR_SVM_time.csv", x, time_run, None, None)
plt.show()
print "done"

['0.10', '0.20', '0.30', '0.40', '0.50', '0.60', '0.70', '0.80', '0.90', '1.00']
[0.6191575091575092, 0.6096223316912972, 0.5987820061075875, 0.6326797194623028, 0.5932279402690362, 0.5976340800488418, 0.59186490503595, 0.5926626104592205, 0.6054533018876128, 0.6006579825562213]


done
